In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
import scipy.signal as signal
import os
%matplotlib inline

In [ ]:
input_folder = "./hybridization_results/"
output_folder = "./hybridization_figures/"
try:
    os.mkdir(output_folder)
except FileExistsError:
    pass

font_scale = 1.75
figsize = (8, 6)
double_figsize = (8, 10)
dpi = 300
handlelength = .8
markersize=7.5
def plot_style():
    sns.set(style='ticks', font_scale=font_scale)
    matplotlib.rcParams['mathtext.fontset'] = 'stix'
    matplotlib.rcParams['font.family'] = 'STIXGeneral'
    matplotlib.rcParams['legend.fontsize'] = 17.5
    matplotlib.rcParams["legend.frameon"] = False
    
colors_2 = ['red', 'blue']
colors_3 = ['indigo', 'darkcyan', 'coral']

def parse_complex(s):
    if " + " in s:
        r, i = s.split(" + ")
        return float(r) + 1j * float(i.split("im")[0])
    elif " - " in s:
        r, i = s.split(" - ")
        return float(r) - 1j * float(i.split("im")[0])

# Scattering

In [ ]:
df = pandas.read_csv(input_folder + "S12.csv")
freqs = np.array(df['freqs'])
circ_S_p5 = np.array([parse_complex(s) for s in df['circ_S_p5']])
circ_S_1p5 = np.array([parse_complex(s) for s in df['circ_S_1p5']])

In [ ]:
plot_style()
fig, ax = plt.subplots(1,1,figsize=figsize)
cp_name = 'x_t'
ax.plot(freqs/1e9, np.abs(circ_S_p5), label=f'${cp_name}$ = 0.5 mm', color=colors_2[0])
ax.plot(freqs/1e9, np.abs(circ_S_1p5), label=f'${cp_name}$ = 1.5 mm', color=colors_2[1])
ax.legend(loc=4, handlelength=handlelength)
ax.set_xlabel('Frequency [GHz]')
ax.set_ylabel(u'$|S_{21}|$')
plt.savefig(output_folder + 'pfilters_s12.png', dpi=dpi)

# Eigenmodes

In [ ]:
df = pandas.read_csv(input_folder + "complex_frequencies.csv")
names = df.columns[0:3]
purcell_filter, resonator_0, resonator_1 = names
names = [resonator_0, resonator_1, purcell_filter] # reorder so purcell filter plots are on top
name_to_color = {purcell_filter: colors_3[0],
                 resonator_0: colors_3[1],
                 resonator_1: colors_3[2]}
name_to_label = {purcell_filter: 'Purcell filter',
                 resonator_0: 'Resonator 0',
                 resonator_1: 'Resonator 1'}
stub_lens = np.array(df['stub_len'])

In [ ]:
plot_style()
fig, ax = plt.subplots(1,1,figsize=figsize)

for name in names:
    ax.semilogy(stub_lens * 1e3, [-2 * parse_complex(s).real/(2*np.pi) for s in df[name]], '.-', markersize=markersize,
         label=name_to_label[name], color=name_to_color[name])

label = ax.xaxis.get_ticklabels()[2]
label.set_bbox(dict(facecolor='none', edgecolor='red', mutation_aspect=.5))
label = ax.xaxis.get_ticklabels()[4]
label.set_bbox(dict(facecolor='none', edgecolor='red', mutation_aspect=.5))

ax.set_ylim([None, 10**(10.5)])
ax.set_xlabel(f'${cp_name}$ [mm]')
ax.set_ylabel('Decay rate [Hz]')
ax.legend(loc=2, handlelength=handlelength)

# INSET
left, bottom, width, height = [0.6, 0.43, 0.27, 0.24]
ax1 = fig.add_axes([left, bottom, width, height])
for name in names:
    ax1.plot(stub_lens * 1e3, [-2 * parse_complex(s).real/(2*np.pi) for s in df[name]], '.-', markersize=markersize,
         label=name_to_label[name], color=name_to_color[name])
ax1.set_xlim([0, 2])
ax1.set_ylim([0, 7e7])
plt.savefig(output_folder + 'loss_rates_vs_stubs.png', dpi=dpi)

In [ ]:
plot_style()
fig, ax = plt.subplots(1,1,figsize=figsize)

for name in names:
    ax.plot(stub_lens * 1e3, [parse_complex(s).imag/(2*np.pi*1e9) for s in df[name]], '.-', markersize=markersize,
         label=name_to_label[name], color=name_to_color[name])

label = ax.xaxis.get_ticklabels()[2]
label.set_bbox(dict(facecolor='none', edgecolor='red', mutation_aspect=.5))
label = ax.xaxis.get_ticklabels()[4]
label.set_bbox(dict(facecolor='none', edgecolor='red', mutation_aspect=.5))

ax.set_ylim([None, 6.07])
ax.set_xlabel(f'${cp_name}$ [mm]')
ax.set_ylabel('Frequency [GHz]')
ax.legend(loc=2, handlelength=handlelength)
plt.savefig(output_folder + 'frequencies_v_stubs.png', dpi=dpi)

# Eigenvectors

In [ ]:
df_p5 = pandas.read_csv(input_folder + "mode_profile_p5.csv")
df_1p5 = pandas.read_csv(input_folder + "mode_profile_1p5.csv")
region_sums = np.cumsum([201+1, 102+1, 101+2]) # see Hybridization.jl
fill_inds = [range(0, region_sums[0]),
             range(region_sums[0]-1, region_sums[1]),
             range(region_sums[1]-1, region_sums[2])]

In [ ]:
plot_style()
transparency = 0.1
fig, axes = plt.subplots(3,1,sharex=True, figsize=double_figsize)
for ax, mode in zip(axes, names):
    ymax = 0.15
    ax.set_ylim([None, ymax])
    colors = ['darkblue', 'darkred']
    for df, c in zip([df_p5, df_1p5], colors):
        y = df[mode]
        x = range(len(y))
        ax.plot(x, y, color=c)
        ax.set_xlim([x[0], x[-1]])
        for i in range(len(names)):
            ax.fill_between(fill_inds[i], ymax, color=name_to_color[names[i]], alpha=transparency)
    ax.set_xticks([])
    ax.set_ylabel(name_to_label[mode])
axes[-1].text(95,-.015, u"$f$")
axes[-1].text(247,-.015, u"$r_0$")
axes[-1].text(352,-.015, u"$r_1$")
axes[-1].legend(ncol=2, bbox_to_anchor=(.62, -0.1), handlelength=handlelength)
plt.tight_layout()
plt.savefig(output_folder + 'hybridization.png', dpi=dpi)

# Hybridization

In [ ]:
df_curves = pandas.read_csv(input_folder + "hybridization_curves.csv")
stub_lens = np.array(df_curves['stub_len'])
df_xs = pandas.read_csv(input_folder + "hybridization_scatter_x.csv")
df_ys = pandas.read_csv(input_folder + "hybridization_scatter_y.csv")

In [ ]:
plot_style()
fig, ax = plt.subplots(1,1,figsize=figsize)

for name in names:
    ax.plot(stub_lens * 1e3, df_curves[name], '.-', markersize=markersize,
         label=name_to_label[name], color=name_to_color[name])

label = ax.xaxis.get_ticklabels()[2]
label.set_bbox(dict(facecolor='none', edgecolor='red', mutation_aspect=.5))
label = ax.xaxis.get_ticklabels()[4]
label.set_bbox(dict(facecolor='none', edgecolor='red', mutation_aspect=.5))

ax.set_xlabel(f'${cp_name}$ [mm]')
ax.set_ylabel('Distance')
ax.legend(loc='best', handlelength=handlelength)

# INSET
left, bottom, width, height = [0.5, 0.25, .4, .4]
ax1 = fig.add_axes([left, bottom, width, height])
sns.despine(ax=ax1, top=True, right=True, left=True, bottom=True)
basis_points = [np.array([0,0]), np.array([1,0]), np.array([1/2, np.sqrt(3)/2])]
xs, ys = zip(*(basis_points + basis_points[:1]))
ax1.plot(xs, ys, c='black')
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_aspect('equal')
for name in names:
    ax1.plot(df_xs[name], df_ys[name], '.-',label=name_to_label[name], color=name_to_color[name])

plt.savefig(output_folder + 'barycentric_overlaps.png', dpi=dpi)

# Convergence

In [ ]:
eigenvalues = np.load(input_folder + "convergence_eigenvalues.npy")
mode_names = ["Purcell filter", "Resonator 0", "Resonator 1"]
def diag_T1(data):
    ans = 1/(-2 * 2 * np.pi * np.array(data).real)
    return ans
def diag_freq(data):
    return np.array(data).imag

In [ ]:
plot_style()
fig, axs = plt.subplots(2, 1, figsize=figsize)

def make_plot(ax, f, results_object, y_min, y_max):
    for i, mode_name in enumerate(mode_names):
        a=f(results_object[i, :, -3])
        b=f(results_object[i, :, -2])
        c=f(results_object[i, :, -1])
        y1 = np.abs((a-c)/c)
        y2 = np.abs((b-c)/c)
        y1_label = mode_name + u' $\delta_2$'
        y2_label = mode_name + " $\delta_1$"
        ax.semilogy(stub_lens * 1e3, y2, markersize=3,
                    label=y2_label, color=colors_3[i])
        ax.semilogy(stub_lens * 1e3, y1, "--", markersize=3,
                    label=y1_label, color=colors_3[i])
        ax.set_ylim([y_min, y_max])
        ax.minorticks_off()

make_plot(axs[0], diag_freq, eigenvalues, .5e-6, 3e-5)
make_plot(axs[1], diag_T1, eigenvalues, .5e-7, 2e-4)

axs[0].set_xticks([])
label = axs[1].xaxis.get_ticklabels()[2]
label.set_bbox(dict(facecolor='none', edgecolor='red', mutation_aspect=.5))
label = axs[1].xaxis.get_ticklabels()[4]
label.set_bbox(dict(facecolor='none', edgecolor='red', mutation_aspect=.5))


axs[0].legend(loc=4, ncol=3, handlelength=handlelength)

axs[0].set_ylabel('Frequency rel. diff.')
axs[1].set_ylabel(u'$T_1$ rel. diff.')
axs[1].set_xlabel(f'${cp_name}$ [mm]')
plt.tight_layout(pad=0.0)
plt.savefig(output_folder + 'hybridization_convergence.png', dpi=dpi)